<a href="https://colab.research.google.com/github/domschl/ALU_Net/blob/main/ALU_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating an ALU (arithmetic logic unit) with a neural network

The neural network is trained to perform the operations `+`, `-`, `*`, `/`, `%`, `AND`, `OR`, `XOR`, `>`, `<`, `=`, `!=` on two unsigned integers and return the result.

## This notebook can run

- on local jupyter instances with a local graphics card
- on Mac M1 with local jupyter instance and [Apple's tensorflow-plugin](https://developer.apple.com/metal/tensorflow-plugin/)
- on Google Colab instances with either GPU or TPU runtime. The colab version uses a Google Drive account to cache data and model state within a Google Drive directory `My Drive/Colab Notebooks/ALU_Net`.

## 1. Configuration and setup

In [1]:
import os
import copy
import json
try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf
import numpy as np

use_keras_project_versions=False
# Namespaces, namespaces
if use_keras_project_versions is False:
    # print("Importing Keras from tensorflow project (it won't work otherwise with TPU)")
    from tensorflow import keras
    from tensorflow.keras import layers, regularizers, callbacks, metrics, optimizers
else:
    # print("Importing Keras from keras project (which had recently declared independence [again]) -- as recommended")
    use_keras_project_versions=True
    import keras
    from keras import layers, regularizers, callbacks, metrics, optimizers

try:
    # Google Drive is used in Colab instances to save trained nets and tensorboard logs
    from google.colab import drive
    is_colab_init = True
except:
    is_colab_init = False
    pass

if is_colab_init is True:
    # The following code loads the utility modules directly from github
    # Into Google Colab (or other jupyter instances)
    # WARNING: indeterministic caching by infrastructure: it might take some
    # minutes until a change in github is actually accessible in colab (aggressive caching). Sometimes
    # it's necessary to factory reset the runtime, and even that gets ignoredd from time to time.
    def import_from_github(fn, repo_root_link, force_github_update=False):
        if os.path.exists(fn) is False or force_github_update is True:
            repo_link=repo_root_link+fn
            print(f"Loading {fn} module from github at {repo_link}...")
            if os.path.exists(fn) is True:
                !rm -v {fn}
            !wget -nv --show-progress {repo_link}
    force_github_update = True  # Note: Even if set to True, you still need to restart the runtime to get an updated version.
    repo_root_link = 'https://raw.githubusercontent.com/domschl/ALU_Net/main/'
    import_from_github('ml_env_tools.py', repo_root_link, force_github_update)
    import_from_github('ml_tuner.py', repo_root_link, force_github_update)
    import_from_github('ALU_Dataset.py', repo_root_link, force_github_update)
    import_from_github('keras_custom_layers.py', repo_root_link, force_github_update)

from ml_env_tools import MLEnv
from ml_tuner import MLTuner
from ALU_Dataset import ALU_Dataset
from keras_custom_layers import ResidualBlock, ResidualDense, ResidualDenseStack, ParallelResidualDenseStacks, SelfAttention, MultiHeadSelfAttention

In [2]:
def model_res_mod(inputs, params):
    # see: keras_custom_layers.py for layer definition:
    x=inputs
    print(f"input-shape: {x.shape}")
    self_att=[]
    for _ in range(0, params['self_attention_layers']):
        self_att.append(MultiHeadSelfAttention(params['self_attention_heads'], units=params['self_attention_units']))
    for i in range(0, params['self_attention_layers']):
        x=self_att[i](x)+x
    fl = layers.Flatten()
    print(f"x.shape bef. fl: {x.shape}")
    x = fl(x)
    print(f"x.shape after. fl: {x.shape}")
    scale = layers.Dense(params['units'], activation=None)
    x=scale(x)
    prds = ResidualDenseStack(params["units"], params["layers"], regularizer=params["regularizer"])
    x=prds(x)
    rescale = layers.Dense(params['output_size'], activation="sigmoid")
    outputs = rescale(x)
    return outputs

In [3]:
def create_load_model(ml_env:MLEnv, model_variant, params, save_path=None, import_weights=True):
    """ Create or load a model """
    if save_path is None or not os.path.exists(save_path) or import_weights is False: #or is_tpu is True:
        print("Initializing new model...")
        # inputs = keras.Input(shape=(params['input_size'],))  # depends on encoding of op-code!
        inputs = keras.Input(shape=params['input_size'])  # depends on encoding of op-code!
        if model_variant not in model_variants:
            print('Unkown model type')
            return None
        outputs = model_variants[model_variant](inputs, params)
        model = keras.Model(inputs=inputs, outputs=outputs, name="maths_"+model_variant)
        print(f"Compiling new model of type {model_variant}")
        if use_keras_project_versions is False: 
            opti = keras.optimizers.Adam(learning_rate=params["learning_rate"])
        else:
            opti = optimizers.Adam(learning_rate=params["learning_rate"])
        if ml_env.is_tpu:
            # use steps_per_execution magic (or not)
            # model.compile(loss="mean_squared_error", optimizer=opti, steps_per_execution=50, metrics=[metrics.MeanSquaredError(), 'accuracy'])
            model.compile(loss="mean_squared_error", optimizer=opti, metrics=[metrics.MeanSquaredError(), 'accuracy'])
        else:
            model.compile(loss="mean_squared_error", optimizer=opti, metrics=[metrics.MeanSquaredError(), 'accuracy'])
    else:
        print(f"Loading standard-format model of type {model_variant} from {model_path}")
        model = tf.keras.models.load_model(save_path)
        print("Continuing training from existing model")
    model.summary()
    return model

In [4]:
def get_model(ml_env, model_variant, params, save_path=None, on_tpu=False, import_weights=False):
    if on_tpu is True:
        if ml_env.tpu_is_init is False:
            cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=ml_env.tpu_address)
            tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
            tpu_strategy = tf.distribute.TPUStrategy(cluster_resolver)    
            ml_env.tpu_is_init=True
        with tpu_strategy.scope():
            print("Creating TPU-scope model")
            model = create_load_model(ml_env, model_variant, params, save_path=save_path, import_weights=import_weights)
        if import_weights is True and ml_env.weights_file is not None and os.path.exists(ml_env.weights_file):
            print("Injecting saved weights into TPU model, loading...")
            temp_model = create_load_model(ml_env, model_variant, params, save_path=save_path, import_weights=import_weights)
            temp_model.load_weights(ml_env.weights_file)
            print("Injecting...")
            model.set_weights(temp_model.get_weights())
            print("Updated TPU weights from saved model")
        return model
    else:
        print("Creating standard-scope model")
        model = create_load_model(ml_env, model_variant, params, save_path=save_path, import_weights=import_weights)
        if import_weights is True and ml_env.weights_file is not None and os.path.exists(ml_env.weights_file):
            print("Injecting saved weights into model, loading...")        
            model.load_weights(ml_env.weights_file)
            imported_weights_file = ml_env.weights_file+'-imported'
            os.rename(ml_env.weights_file, imported_weights_file)
            print(f"Renamed weights file {ml_env.weights_file} to {imported_weights_file} to prevent further imports!")
        return model

In [5]:
def math_train(mlenv:MLEnv, model, dataset, validation, batch_size=8192, epochs=5000, steps_per_epoch=2000, log_path="./logs"):
    """ Training loop """
    interrupted = 2
    hist = None
    tensorboard_callback = callbacks.TensorBoard(
        log_dir=log_path
        # histogram_freq=1
        # update_freq='batch'
        )
    if mlenv.is_tpu is False: # TPUs update Tensorboard too asynchronously, data is corrupted by updates during mirroring.
        lambda_callback = tf.keras.callbacks.LambdaCallback(
            on_epoch_end = ml_env.epoch_time_func
        )
    try:
        if ml_env.is_tpu:
            if use_validation_with_tpu is True:
                hist = model.fit(dataset, validation_data=validation, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, verbose=1, callbacks=[tensorboard_callback])
            else:
                hist = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[tensorboard_callback])
            interrupted=0
        else:
            hist = model.fit(dataset, validation_data=validation, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[tensorboard_callback, lambda_callback])
            interrupted=0
    except KeyboardInterrupt:
        print("")
        print("")
        print("---------INTERRUPT----------")
        print("")
        print("Training interrupted")
        interrupted = 1 # user stopped runtime
    except Exception as e:
        interruped = 2  # Bad: something crashed.
        print(f"INTERNAL ERROR")
        print(f"Exception {e}")
    finally:
        return interrupted, hist

In [6]:
def instantiate_models(ml_env:MLEnv, model_variant, params, save_path=None, import_weights=True):
    if ml_env.is_tpu:
        # Generate a second CPU model for testing:
        math_model = get_model(ml_env, model_variant, params, save_path=save_path, on_tpu=True, import_weights=import_weights)
        test_model = get_model(ml_env, model_variant, params, save_path=save_path, on_tpu=False, import_weights=import_weights)
    else:
        test_model = None
        math_model = get_model(ml_env, model_variant, params, save_path=save_path, on_tpu=False, import_weights=import_weights)
    return math_model, test_model

In [7]:
def do_training(mlenv:MLEnv, math_model, training_dataset, validation_dataset, math_data, epochs_per_cycle, model_path=None, 
                weights_file=None, test_model=None, cycles=100, steps_per_epoch=1000, reweight_size=1000, valid_ops=None, regenerate_data_after_cycles=0, data_func=None,
                log_path='./logs'):
    # Training
    for mep in range(0, cycles):
        print()
        print()
        print(f"------ Meta-Epoch {mep+1}/{cycles} ------")
        print()
        if regenerate_data_after_cycles!=0 and data_func is not None:
            if mep>0 and (mep+1)%regenerate_data_after_cycles==0:
                training_dataset, validation_dataset = data_func()
        if mep==0 and ml_env.is_tpu is True:
            print("There will be some warnings by Tensorflow, documenting some state of internal decoherence, currently they can be ignored.")
        interrupted, hist = math_train(ml_env, math_model, training_dataset, validation=validation_dataset, epochs=epochs_per_cycle, steps_per_epoch=steps_per_epoch, log_path=log_path)
        if interrupted <2:
            if ml_env.is_tpu:
                mlenv.gdrive_log_mirror()  # TPUs can only savely mirror Tensorboard data once training is finished for an meta-epoch.
                if test_model is None:
                    print("Fatal: tpu-mode needs test_model on CPU")
                    return False
                print("Injecting weights into test_model:")
                test_model.set_weights(math_model.get_weights())
                if weights_file is not None:
                    print(f"Saving test-model weights to {weights_file}")
                    test_model.save_weights(weights_file)
                    print("Done")
                print(f"Checking {reweight_size} datapoints for accuracy...")
                math_data.check_results(test_model, samples=reweight_size, vector=vector, valid_ops=valid_ops, verbose=False)
            else:
                if model_path is not None:
                    print("Saving math-model")
                    math_model.save(model_path)
                    print("Done")
                print(f"Checking {reweight_size} datapoints for accuracy...")
                math_data.check_results(math_model, samples=reweight_size, vector=vector, valid_ops=valid_ops, verbose=False)
        if interrupted>0:
            break

In [8]:
ml_env=MLEnv()
bit_count = 15
math_data=ALU_Dataset(ml_env, bit_count = bit_count)

CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')] {}
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] {'device_name': 'METAL'}
GPU available


In [9]:
model_variants = {"res_mod": model_res_mod,
                  }

model_variant = 'res_mod'  # see: model_variants definition.
epochs_per_cycle = 100
cycles = 100  # perform 100 (meta-)cycles, each cycle trains with epochs_per_cycle epochs.
regenerate_data_after_cycles = 0  # if !=0, the training data will be created anew after each number of 
                                  # regenerace_data_after_cycles cycles. Disadvantage: when training TPU, 
                                  # Google might use the time it takes to regenerate to training data to 
                                  # terminate your session :-/
low_resource = True

if low_resource is True:
    samples = 100000  # Number training data examples. 
                    # WARNING: TPU simply crashes, if 2GB limit for entire set is reached.
                    # Possible solutions: https://www.tensorflow.org/api_docs/python/tf/data/experimental/service#running_the_tfdata_service,
                    # https://www.tensorflow.org/api_docs/python/tf/data/experimental/service , https://github.com/tensorflow/models/blob/master/official/recommendation/ncf_input_pipeline.py#L33
    validation_samples=10000
else:
    samples = 4000000  # Number training data examples. 
                    # WARNING: TPU simply crashes, if 2GB limit for entire set is reached.
                    # Possible solutions: https://www.tensorflow.org/api_docs/python/tf/data/experimental/service#running_the_tfdata_service,
                    # https://www.tensorflow.org/api_docs/python/tf/data/experimental/service , https://github.com/tensorflow/models/blob/master/official/recommendation/ncf_input_pipeline.py#L33
    validation_samples=100000
    
if low_resource is True:
    batch_size = 2000
else:
    batch_size = 20000
import_weights=True
if import_weights is False:
    print("WARNING: import weights is set to False!")
valid_ops = None  # Default: None (all ops), or list of ops, e.g. ['*', '/'] trains only multiplication and division.
# valid_ops = ['*','/','+','-']
# valid_ops = ['*']
steps_per_epoch = samples // batch_size  # TPU stuff
validation_steps= validation_samples // batch_size  # again TPU only
use_validation_with_tpu = False  # Is somehow really, really slow

params_res_mod={
    "self_attention_layers": 4,
    "self_attention_heads": 4,
    "self_attention_units": 64,
    "units": 64,
    "learning_rate": 0.002,
    "layers": 1,
    "regularizer": 1e-9
    }

params=params_res_mod
vector = True

if vector is True:
    params['input_size'] = [3, math_data.embedding_size]
else:
    params['input_size'] = math_data.input_size
params['output_size'] = math_data.output_size

In [10]:
root_path, project_path, model_path, weights_file, cache_path, log_path = ml_env.init_paths("ALU_Net", "math_model", model_variant=model_variant, log_to_gdrive=False)

Root path: .
Model save-path: ./math_model_res_mod
Data cache path ./data


In [11]:
apply_model_tuner = False   # Use GPU (not TPU!) for model_tuner.

In [12]:
if apply_model_tuner is True:
    as_train, as_val = math_data.get_datasets(samples=500000, validation_samples=50000, vector=vector, cache_path=cache_path)

    def tuner_eval(ml_env:MLEnv, model_variant, params, batch_size, epochs):
        math_model, _ = instantiate_models(ml_env, model_variant, params, save_path=None, import_weights=False)
        interrupted, hist = math_train(ml_env, math_model, as_train, as_val, batch_size=batch_size, epochs=epochs)
        print(params, end=" [ ")
        res = math_data.check_results(math_model, samples=100, valid_ops=valid_ops, verbose=False)
        ev = 1/hist.history['val_loss'][-1]+hist.history['val_accuracy'][-1]*20
        if res>0:
            print("Success-rate: {res}")
            ev += res*5000
        return ev

    tuner_eval_func = lambda params : tuner_eval(ml_env, model_variant, params, batch_size=batch_size, epochs=20)
    ml_tuner = MLTuner(ml_env, model_variant)

    param_space_minimal_prm={
    "dense_layers": [4,8,12],
    "dense_neurons":[256,512,768], 
    "learning_rate": [0.001,0.002],
    "regu1": [1e-8,1e-7]
    }

    best_params = ml_tuner.tune(param_space_minimal_prm, tuner_eval_func)
    params = best_params
    import_weights=False

In [13]:
params

{'self_attention_layers': 4,
 'self_attention_heads': 4,
 'self_attention_units': 64,
 'units': 64,
 'learning_rate': 0.002,
 'layers': 1,
 'regularizer': 1e-09,
 'input_size': [3, 16],
 'output_size': 32}

In [14]:
create_train_val_data = lambda regen : math_data.get_datasets(pre_weight=False, samples=samples, validation_samples=validation_samples, batch_size=batch_size, 
                                     vector=vector, valid_ops=valid_ops, cache_path=cache_path, use_cache=True, regenerate_cached_data=regen)
create_train_val_data_regen = lambda : create_train_val_data(True)
train, val = create_train_val_data(False)

Data train loaded from cache
Metal device set to: Apple M1
Data validation loaded from cache


2021-12-07 07:58:15.091083: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 07:58:15.091188: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# !pip install tensorflow_datasets
# import tensorflow_datasets as tdfs
# sa=SelfAttention()
# nval=tdfs.as_numpy(val)
# for n in nval:
#     print(n[0].shape)   
#     print(sa(n[0]).shape)
#     break

In [16]:
math_model, test_model = instantiate_models(ml_env, model_variant, params, save_path=model_path, import_weights=import_weights)
# math_model, test_model = instantiate_models(ml_env, model_variant, params, save_path=None, import_weights=False)

Creating standard-scope model
Initializing new model...
input-shape: (None, 3, 16)
x.shape bef. fl: (None, 3, 16)
x.shape after. fl: (None, 48)
Compiling new model of type res_mod
Model: "maths_res_mod"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 16)]      0                                            
__________________________________________________________________________________________________
multi_head_self_attention (Mult (None, 3, 16)        3172        input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.add (TFOpLambd (None, 3, 16)        0           multi_head_self_attention[0][0]  
                                                                 input_1[0][0]                    
_____

In [17]:
try:
    # use the python variable log_path:
    get_ipython().run_line_magic('tensorboard', '--logdir "{log_path}"')
except:
    pass

In [ ]:
do_training(ml_env, math_model, train, val, math_data, epochs_per_cycle, model_path=model_path, 
            weights_file=weights_file, test_model=test_model, cycles=cycles, steps_per_epoch=steps_per_epoch, valid_ops=valid_ops, 
            regenerate_data_after_cycles=regenerate_data_after_cycles, data_func=create_train_val_data_regen, log_path=log_path)



------ Meta-Epoch 1/100 ------

Epoch 1/100


2021-12-07 07:58:15.530261: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-07 07:58:15.530273: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-07 07:58:15.530447: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-07 07:58:16.785490: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-07 07:58:16.795829: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-07 07:58:16.796207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 1/50 [..............................] - ETA: 1:48 - loss: 0.3196 - mean_squared_error: 0.3196 - accuracy: 0.0390

2021-12-07 07:58:17.832158: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-07 07:58:17.832170: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/50 [>.............................] - ETA: 14s - loss: 0.3118 - mean_squared_error: 0.3118 - accuracy: 0.0383 

2021-12-07 07:58:18.081128: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-07 07:58:18.104279: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-07 07:58:18.123342: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/train/plugins/profile/2021_12_07_07_58_18

2021-12-07 07:58:18.139985: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2021_12_07_07_58_18/m1air.fritz.box.trace.json.gz
2021-12-07 07:58:18.145285: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/train/plugins/profile/2021_12_07_07_58_18

2021-12-07 07:58:18.145489: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to ./logs/train/plugins/profile/2021_12_07_07_58_18/m1air.fritz.box.memory_profile.json.gz
2021-12-07 07:5

50/50 [==============================] - ETA: 0s - loss: 0.1350 - mean_squared_error: 0.1350 - accuracy: 0.1059

2021-12-07 07:58:30.312034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 15s 266ms/step - loss: 0.1350 - mean_squared_error: 0.1350 - accuracy: 0.1059 - val_loss: 0.0958 - val_mean_squared_error: 0.0958 - val_accuracy: 0.1962
Epoch 2/100
50/50 [==============================] - 13s 253ms/step - loss: 0.0746 - mean_squared_error: 0.0746 - accuracy: 0.1709 - val_loss: 0.0761 - val_mean_squared_error: 0.0761 - val_accuracy: 0.0813
Epoch 3/100
50/50 [==============================] - 13s 253ms/step - loss: 0.0676 - mean_squared_error: 0.0676 - accuracy: 0.2031 - val_loss: 0.0780 - val_mean_squared_error: 0.0780 - val_accuracy: 0.1098
Epoch 4/100
38/50 [=====================>........] - ETA: 2s - loss: 0.0646 - mean_squared_error: 0.0646 - accuracy: 0.2243

# Testing and applying the trained model

In [ ]:
if ml_env.is_tpu is False:
    test_model = math_model
math_data.check_results(test_model, samples=100, valid_ops=valid_ops, verbose=True)

In [ ]:
dx,dy,_,_,_=math_data.create_data_point(22,33,'*'); print(22*33)

In [ ]:
r=test_model.predict(np.array([dx]))
print(r)
math_data.decode_results(r)

In [ ]:
def calc(inp):
    args=inp.split(' ')
    if len(args)!=3:
        print("need three space separated tokens: <int> <operator> <int>, e.g. '3 + 4' or '4 XOR 5'")
        return False
    if args[1] not in math_data.model_ops:
        print(f"{args[1]} is not a known operator.")
        return False
    op1=int(args[0])
    op2=int(args[2])
    dx,dy,_,_,_=math_data.create_data_point(op1, op2, args[1])
    ans=math_data.decode_results(test_model.predict(np.array([dx])))
    print(f"{op1} {args[1]} {op2} = {ans[0]}")
    op=f"{op1} {args[1]} {op2}"
    op=op.replace('AND', '&').replace('XOR','^').replace('=','==').replace('OR','|')
    an2=eval(op)
    if ans[0]!=an2:
        print("Error")
        print(bin(ans[0]))
        print(bin(an2))
    return ans[0],an2

In [ ]:
calc("22 * 33")

In [ ]:
calc("1 = 1")

In [ ]:
calc("3 * 4")

In [ ]:
calc ("1 AND 3")